In [4]:
import os
import fnmatch
from textblob import TextBlob
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import pos_tag,pos_tag_sents
import regex as re
import operator
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import pickle
from nltk.corpus import stopwords

In [6]:
path = './op_spam_train/'

label = []

configfiles = [os.path.join(subdir,f)
for subdir, dirs, files in os.walk(path)
    for f in fnmatch.filter(files, '*.txt')]

In [7]:
len(configfiles)

1600

In [8]:
configfiles[1]

'./op_spam_train/positive_polarity/deceptive_from_MTurk/fold2/d_talbott_8.txt'

In [9]:
for f in configfiles:
    c = re.search('(trut|deceptiv)\w',f)
    label.append(c.group())


In [10]:
labels = pd.DataFrame(label, columns = ['Labels'])

In [11]:
labels.head(5)

,Labels
0,deceptive
1,deceptive
2,deceptive
3,deceptive
4,deceptive


In [12]:
review = []
directory =os.path.join("./op_spam_train/")
for subdir,dirs ,files in os.walk(directory):
   # print (subdir)
    for file in files:
        if fnmatch.filter(files, '*.txt'):
            f=open(os.path.join(subdir, file),'r')
            a = f.read()
            review.append(a)

In [13]:
reviews = pd.DataFrame(review, columns = ['HotelReviews'])

In [14]:
reviews.head(5)

,HotelReviews
0,"excellent staff and customer service, very cle..."
1,My stay at this hotel was one of the best I ha...
2,We just got back from a trip to Chicago for my...
3,I have to say that the Hard Rock Hotel in Chic...
4,My husband and I recently stayed at the Hard R...


In [15]:
result = pd.merge(reviews, labels,right_index=True,left_index = True)
result['HotelReviews'] = result['HotelReviews'].map(lambda x: x.lower())

In [16]:
result.to_csv('Exaggeration in fake vs. authentic online reviews for luxury and budget hotels .csv')

In [17]:
result.head()

,HotelReviews,Labels
0,"excellent staff and customer service, very cle...",deceptive
1,my stay at this hotel was one of the best i ha...,deceptive
2,we just got back from a trip to chicago for my...,deceptive
3,i have to say that the hard rock hotel in chic...,deceptive
4,my husband and i recently stayed at the hard r...,deceptive


In [18]:
pip install --user -U nltk

Note: you may need to restart the kernel to use updated packages.


In [19]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prashanthreddy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
stop = stopwords.words('english')
result['review_without_stopwords'] = result['HotelReviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [22]:
result.head()

,HotelReviews,Labels,review_without_stopwords
0,"excellent staff and customer service, very cle...",deceptive,"excellent staff customer service, clean spotle..."
1,my stay at this hotel was one of the best i ha...,deceptive,"stay hotel one best ever had! location, servic..."
2,we just got back from a trip to chicago for my...,deceptive,got back trip chicago 30th birthday could impr...
3,i have to say that the hard rock hotel in chic...,deceptive,say hard rock hotel chicago cool place stay. f...
4,my husband and i recently stayed at the hard r...,deceptive,husband recently stayed hard rock hotel chicag...


In [23]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/prashanthreddy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/prashanthreddy/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [24]:
def pos(review_without_stopwords):
    return TextBlob(review_without_stopwords).tags

In [25]:
os = result.review_without_stopwords.apply(pos)
os1 = pd.DataFrame(os)

In [26]:
os1.head()

,review_without_stopwords
0,"[(excellent, JJ), (staff, NN), (customer, NN),..."
1,"[(stay, JJ), (hotel, NN), (one, CD), (best, JJ..."
2,"[(got, VBD), (back, RB), (trip, NN), (chicago,..."
3,"[(say, VB), (hard, JJ), (rock, NN), (hotel, NN..."
4,"[(husband, NN), (recently, RB), (stayed, VBD),..."


In [27]:
os1['pos'] = os1['review_without_stopwords'].map(lambda x:" ".join(["/".join(x) for x in x ]) )

In [28]:
result = result = pd.merge(result, os1,right_index=True,left_index = True)
result.head()

,HotelReviews,Labels,review_without_stopwords_x,review_without_stopwords_y,pos
0,"excellent staff and customer service, very cle...",deceptive,"excellent staff customer service, clean spotle...","[(excellent, JJ), (staff, NN), (customer, NN),...",excellent/JJ staff/NN customer/NN service/NN c...
1,my stay at this hotel was one of the best i ha...,deceptive,"stay hotel one best ever had! location, servic...","[(stay, JJ), (hotel, NN), (one, CD), (best, JJ...",stay/JJ hotel/NN one/CD best/JJS ever/RB had/V...
2,we just got back from a trip to chicago for my...,deceptive,got back trip chicago 30th birthday could impr...,"[(got, VBD), (back, RB), (trip, NN), (chicago,...",got/VBD back/RB trip/NN chicago/NN 30th/CD bir...
3,i have to say that the hard rock hotel in chic...,deceptive,say hard rock hotel chicago cool place stay. f...,"[(say, VB), (hard, JJ), (rock, NN), (hotel, NN...",say/VB hard/JJ rock/NN hotel/NN chicago/NN coo...
4,my husband and i recently stayed at the hard r...,deceptive,husband recently stayed hard rock hotel chicag...,"[(husband, NN), (recently, RB), (stayed, VBD),...",husband/NN recently/RB stayed/VBD hard/JJ rock...


In [29]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=13)

In [30]:
tf_vect = TfidfVectorizer(lowercase = True, use_idf=True, smooth_idf=True, sublinear_tf=False)

X_train_tf = tf_vect.fit_transform(review_train)
X_test_tf = tf_vect.transform(review_test)

In [31]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='linear'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    return grid_search.best_params_

In [32]:
svc_param_selection(X_train_tf,label_train,5)

{'C': 1, 'gamma': 0.001}

In [33]:
{'C': 10, 'gamma': 0.001}

{'C': 10, 'gamma': 0.001}

In [34]:
clf = svm.SVC(C=10,gamma=0.001,kernel='linear')
clf.fit(X_train_tf,label_train)
pred = clf.predict(X_test_tf)

In [35]:
with open('vectorizer.pickle', 'wb') as fin:
    pickle.dump(tf_vect, fin)

In [36]:
with open('mlmodel.pickle','wb') as f:
    pickle.dump(clf,f)

In [37]:
pkl = open('mlmodel.pickle', 'rb')
clf = pickle.load(pkl)   
vec = open('vectorizer.pickle', 'rb')
tf_vect = pickle.load(vec)

In [38]:
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

In [39]:
print(metrics.accuracy_score(label_test, pred))

0.86875


In [40]:
print (confusion_matrix(label_test, pred))

[[140  15]
 [ 27 138]]


In [41]:
print (classification_report(label_test, pred))

              precision    recall  f1-score   support

   deceptive       0.84      0.90      0.87       155
       truth       0.90      0.84      0.87       165

    accuracy                           0.87       320
   macro avg       0.87      0.87      0.87       320
weighted avg       0.87      0.87      0.87       320



## Test the Model using different Random States

### Random State 1

In [42]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=1)

In [43]:
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

In [44]:
print(metrics.accuracy_score(label_test, pred))

0.9625


In [45]:
print (confusion_matrix(label_test, pred))

[[146   5]
 [  7 162]]


In [46]:
print (classification_report(label_test, pred))

              precision    recall  f1-score   support

   deceptive       0.95      0.97      0.96       151
       truth       0.97      0.96      0.96       169

    accuracy                           0.96       320
   macro avg       0.96      0.96      0.96       320
weighted avg       0.96      0.96      0.96       320



### Random State 10


In [47]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=10)

In [48]:
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

In [49]:
print(metrics.accuracy_score(label_test, pred))

0.978125


In [50]:
print (confusion_matrix(label_test, pred))

[[156   4]
 [  3 157]]


In [51]:
print (classification_report(label_test, pred))

              precision    recall  f1-score   support

   deceptive       0.98      0.97      0.98       160
       truth       0.98      0.98      0.98       160

    accuracy                           0.98       320
   macro avg       0.98      0.98      0.98       320
weighted avg       0.98      0.98      0.98       320



In [56]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=42)

In [57]:
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

In [58]:
print(metrics.accuracy_score(label_test, pred))

0.971875


In [71]:
print (confusion_matrix(label_test, pred))

[[164   4]
 [  5 147]]


In [72]:
print (classification_report(label_test, pred))

              precision    recall  f1-score   support

   deceptive       0.97      0.98      0.97       168
       truth       0.97      0.97      0.97       152

    accuracy                           0.97       320
   macro avg       0.97      0.97      0.97       320
weighted avg       0.97      0.97      0.97       320



## Test the model with two reviews 

In [73]:
def test_review(s):
    X_test_tf = tf_vect.transform([s])
    y_predict = clf.predict(X_test_tf)
    return y_predict

In [68]:
test_review("The hotel was good.The room had a 27-inch Samsung led tv, a microwave.The room had a double bed")

array(['truth'], dtype=object)

In [69]:
test_review("the experince at the hard rock hotel in chicago was fantastic,i will rate them a 6 out of 5. they have wonderful service and great staff and the view is just wonderful.")

array(['deceptive'], dtype=object)

In [ ]:
`